In [2]:
!git clone https://github.com/lucolivi/dados-emprestimo.git

Cloning into 'dados-emprestimo'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11 (delta 1), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), 81.92 KiB | 1.71 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [3]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential
from azure.ai.ml import automl, Input, MLClient, command
from azure.ai.ml.automl import classification

In [4]:
ml_client = MLClient.from_config(DefaultAzureCredential())

my_data = Data(
    path="./dados-emprestimo",
    type="mltable",
    description="Conjunto de dados sobre realização de empréstimos.",
    name="dados-emprestimo",
)

ml_client.data.create_or_update(my_data)

Found the config file in: /config.json
Uploading dados-emprestimo (0.5 MBs): 100%|██████████| 498711/498711 [00:00<00:00, 1671271.91it/s]




Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': ['./loan_approval_dataset.csv'], 'type': 'mltable', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'dados-emprestimo', 'description': 'Conjunto de dados sobre realização de empréstimos.', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/507fb825-19a0-4a3f-8e46-2a5de5060508/resourceGroups/lucas.oliveira.fiap-rg/providers/Microsoft.MachineLearningServices/workspaces/meu_workspace_1/data/dados-emprestimo/versions/2', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/minhamaquina/code/Users/lucas.oliveira.fiap', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7fb5391e00a0>, 'serialize': <msrest.serialization.Serializer object at 0x7fb5391e12d0>, 'version': '2', 'latest_version': None, 'path': 'azureml://subscriptions/507fb825-19a0-4a3f-8e46-2a5de5060508/resourcegroups/l

In [6]:
classification_job = automl.classification(
    compute="meucluster",
    experiment_name="emprestimo-automl",
    training_data=Input(type="mltable", path="azureml:dados-emprestimo:1"),
    target_column_name="loan_status",
    primary_metric="accuracy",
    n_cross_validations=5
)

In [8]:
classification_job.set_limits(
    timeout_minutes=600,
    trial_timeout_minutes=20,
    max_trials=5,
    enable_early_termination=True,
    max_concurrent_trials=3
)

In [9]:
ml_client.jobs.create_or_update(classification_job)

Experiment,Name,Type,Status,Details Page
emprestimo-automl,joyful_parrot_yxtsprkvcc,automl,NotStarted,Link to Azure Machine Learning studio
